# TM10007 Assignment

In [1]:
# Run this to use from colab environment
!pip install -q --upgrade git+https://github.com/jveenland/tm10007_ml.git

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 54.6 MB/s eta 0:00:00


## Import packages

In [2]:
import pandas as pd

from worcgist.load_data import load_data

## 1. Data loading

Below are functions to load the dataset of your choice. After that, it is all up to you to create and evaluate a classification method. Beware, there may be missing values in these datasets. Good luck!

In [3]:
# Load data
data = load_data()

data['label'] = data['label'].map({'non-GIST': 0, 'GIST': 1})

y = data['label']
x = data.drop(columns=['label'])

# Calculate basic statistics
n_samples = len(y)
n_features = len(x.columns)

n_missing = data.isna().sum().sum()
n_zero = (data == 0).sum().sum()

n_non_gist = (y == 0).sum().sum()
n_gist = (y == 1).sum().sum()
# Print basic statistics
print(f'The number of samples: {n_samples}')
print(f'The number of features: {n_features}')
print(f'The number of datapoints: {n_samples * n_features}')
print(f"The number of missing values: {n_missing}")
print(f"The number of zero values: {n_zero}")
print(f"The percentage of zero values: {n_non_gist/(n_samples*n_features)*100} %")
print(f"The percentage of GIST tumors: {n_gist/n_samples*100} %")
print(f"The percentage of non-GIST tumors: {n_non_gist/n_samples*100} %")
#data.info()
#data.describe()
#data.duplicated()

The number of samples: 246
The number of features: 493
The number of datapoints: 121278
The number of missing values: 0
The number of zero values: 6848
The percentage of zero values: 0.0997707745840136 %
The percentage of GIST tumors: 50.81300813008131 %
The percentage of non-GIST tumors: 49.1869918699187 %


## 2. Pre-processing

### Remove constants

In [4]:
# Remove columns with constant values
constant_cols = x.columns[x.nunique() <= 1] # get columns with 1 unique value or all values NaN
x = x.drop(columns=constant_cols) # drop these columns

# Remove rows with constant values
constant_rows = x.index[x.nunique(axis=1) <= 1] # get rows with 1 unique value or all values NaN
x_clean = x.drop(index=constant_rows) # drop these columns

# Adjust y to match the remainng samples
y_clean = y.drop(index=constant_rows)

# Combine x_clean and y_clean to one DataFrame
data_constants_removed = pd.concat([x_clean, y_clean], axis=1)

# Get dimensions
dimensions_data_constants_removed = data_constants_removed.shape
n_samples_constants_removed, n_features_constants_removed = dimensions_data_constants_removed

# Change in dimension
samples_removed = n_samples - n_samples_constants_removed
features_removed = n_features - n_features_constants_removed

# Result
print(f'Number of removed samples: {samples_removed}')
print(f'Number of removed features: {features_removed}')
print(f'Dataset without constant values: {data_constants_removed.shape}')

data_constants_removed.head()

Number of removed samples: 0
Number of removed features: 12
Dataset without constant values: (246, 481)


,PREDICT_original_sf_compactness_avg_2.5D,PREDICT_original_sf_compactness_std_2.5D,PREDICT_original_sf_rad_dist_avg_2.5D,PREDICT_original_sf_rad_dist_std_2.5D,PREDICT_original_sf_roughness_avg_2.5D,PREDICT_original_sf_roughness_std_2.5D,PREDICT_original_sf_convexity_avg_2.5D,PREDICT_original_sf_convexity_std_2.5D,PREDICT_original_sf_cvar_avg_2.5D,PREDICT_original_sf_cvar_std_2.5D,...,PREDICT_original_phasef_phasesym_mean_WL3_N5,PREDICT_original_phasef_phasesym_median_WL3_N5,PREDICT_original_phasef_phasesym_std_WL3_N5,PREDICT_original_phasef_phasesym_skewness_WL3_N5,PREDICT_original_phasef_phasesym_kurtosis_WL3_N5,PREDICT_original_phasef_phasesym_range_WL3_N5,PREDICT_original_phasef_phasesym_energy_WL3_N5,PREDICT_original_phasef_phasesym_quartile_range_WL3_N5,PREDICT_original_phasef_phasesym_entropy_WL3_N5,label
ID,,,,,,,,,,,,,,,,,,,,,
GIST-001_0,0.797427,0.120594,80.192445,9.445741,7.102210,3.876595,0.927623,0.081385,0.015227,0.010465,...,0.005697,0.0,0.032461,7.353902,63.183181,0.123189,1591.133826,0.0,15.687095,1
GIST-002_0,0.896991,0.045854,35.323607,2.781343,5.339906,1.923100,0.975762,0.017550,0.009118,0.009385,...,0.009552,0.0,0.043431,6.452863,52.634005,0.172040,220.543010,0.0,12.664845,0
GIST-003_0,0.893238,0.051894,17.582541,1.864908,7.622507,1.039745,0.979608,0.017926,0.013258,0.010217,...,0.022104,0.0,0.068175,3.849719,16.277217,0.286710,62.150817,0.0,10.432943,1
GIST-004_0,0.914027,0.032078,21.951056,2.188093,4.757274,1.786309,0.981125,0.013734,0.011338,0.005147,...,0.010130,0.0,0.044148,5.725807,38.600415,0.182812,39.375540,0.0,10.194878,0
GIST-005_0,0.659747,0.266226,20.801224,4.025831,14.404912,7.695865,0.865275,0.171213,0.048078,0.047425,...,0.012287,0.0,0.043369,4.546357,23.831176,0.183349,24.192700,0.0,10.066832,0


In [5]:
from scipy.stats import shapiro

def check_normal_distribution(df, alpha=0.05):
    """
    Check if each column in the given DataFrame follows a normal distribution.

    Parameters:
    df (pd.DataFrame): The input DataFrame containing numerical data.
    alpha (float): The significance level for the Shapiro-Wilk test.

    Returns:
    pd.DataFrame: A DataFrame indicating whether each column in the input DataFrame follows a normal distribution.
    The DataFrame has one row and the same number of columns as the input DataFrame.
    """
    results = {}
    for col in df.columns:
        p = shapiro(df[col])[1]
        results[col] = p > alpha
    return pd.DataFrame([results])

results = check_normal_distribution(x)
#print(results)

In [6]:
def replace_outliers(df, is_normal):
    """
    Replace outliers in the given DataFrame based on the distribution type of each column.

    Parameters:
    df (pd.DataFrame): The input DataFrame containing numerical data.
    is_normal (pd.Series): A Series indicating whether each column in the DataFrame is normally distributed (True) or not (False).

    Returns:
    pd.DataFrame: The DataFrame with outliers replaced by the respective bounds.
    list: A list of all outlier values found.
    """
    threshold_nd = 1  # z-score threshold for normally distributed data
    threshold_nnd = 5  # IQR threshold for non-normally distributed data
    outliers_list = []  # Initialize an empty list to store outliers

    for column in df.columns:
        if is_normal[column]:
            # Use z-score for normally distributed columns
            mean = df[column].mean()
            std = df[column].std()
            lower_bound = mean - threshold_nd * std
            upper_bound = mean + threshold_nd * std
            outliers = (df[column] > upper_bound) | (df[column] < lower_bound)
            outliers_list.extend(df.loc[outliers, column].tolist())  # Append outliers to the list
            df.loc[df[column] > upper_bound, column] = upper_bound
            df.loc[df[column] < lower_bound, column] = lower_bound
        else:
            # Use IQR for non-normally distributed columns
            Q1 = df[column].quantile(0.25)
            Q3 = df[column].quantile(0.75)
            IQR = Q3 - Q1
            lower_bound = Q1 - threshold_nnd * IQR
            upper_bound = Q3 + threshold_nnd * IQR
            outliers = (df[column] > upper_bound) | (df[column] < lower_bound)
            outliers_list.extend(df.loc[outliers, column].tolist())  # Append outliers to the list
            df.loc[df[column] > upper_bound, column] = upper_bound
            df.loc[df[column] < lower_bound, column] = lower_bound

    return df, outliers_list  # Return the modified DataFrame and the outliers list
df_no_outliers, outliers = replace_outliers(x, results.iloc[0])
print("Max of original data (first 5 columns):")
print(x.iloc[:, 10:20].describe().loc['max'])
print("Max of data without outliers (first 5 columns):")
print(df_no_outliers.iloc[:, 10:20].describe().loc['max'])
print("Number of outliers:", len(outliers))

<ipython-input-6-56050a730009>:37: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '87.25' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[df[column] > upper_bound, column] = upper_bound
<ipython-input-6-56050a730009>:37: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '75.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[df[column] > upper_bound, column] = upper_bound
<ipython-input-6-56050a730009>:26: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '28.18683538209948' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[df[column] > upper_bound, column] = upper_bound
<ipython-input-6-56050a730009>:26: FutureWar

Max of original data (first 5 columns):
PREDICT_original_sf_prax_avg_2.5D            0.816314
PREDICT_original_sf_prax_std_2.5D            0.169415
PREDICT_original_sf_evar_avg_2.5D            0.092291
PREDICT_original_sf_evar_std_2.5D            0.056980
PREDICT_original_sf_solidity_avg_2.5D        1.247305
PREDICT_original_sf_solidity_std_2.5D        0.230570
PREDICT_original_sf_area_avg_2.5D        21935.647267
PREDICT_original_sf_area_max_2.5D        36947.705399
PREDICT_original_sf_area_min_2.5D         1895.417736
PREDICT_original_sf_area_std_2.5D        10747.729719
Name: max, dtype: float64
Max of data without outliers (first 5 columns):
PREDICT_original_sf_prax_avg_2.5D            0.816314
PREDICT_original_sf_prax_std_2.5D            0.169415
PREDICT_original_sf_evar_avg_2.5D            0.092291
PREDICT_original_sf_evar_std_2.5D            0.056980
PREDICT_original_sf_solidity_avg_2.5D        1.247305
PREDICT_original_sf_solidity_std_2.5D        0.230570
PREDICT_original_sf_ar

## 3. Feature selection

In [7]:
from sklearn.feature_selection import SelectKBest, f_classif

selector = SelectKBest(score_func=f_classif, k=50)
x_reduced_array = selector.fit_transform(x, y)
selected_feature_names = x.columns[selector.get_support()]
x_reduced = pd.DataFrame(x_reduced_array, columns=selected_feature_names)

/usr/local/lib/python3.11/dist-packages/sklearn/feature_selection/_univariate_selection.py:111: UserWarning: Features [ 76  82  86  88  92  93  97 104 106 315 328 408 415 416 421 428 429 434
 441 442 462 472 478] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/usr/local/lib/python3.11/dist-packages/sklearn/feature_selection/_univariate_selection.py:112: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


## 4. Model training

1. Data Splitting:
   - The dataset is partitioned into an 80% training/validation set and a 20% test set using stratified sampling to preserve the class distribution.
   - The test set is reserved for final evaluation, ensuring an unbiased estimate of model performance on unseen data.

2. Pipeline Construction:
   - For each classifier (SVM, Random Forest, kNN, Logistic Regression, Linear Discriminant Analysis (LDA), and Quadratic Discriminant Analysis (QDA)), a pipeline is defined.
   - Each pipeline integrates a feature scaling step (using StandardScaler) to standardize the features, followed by the classifier.

3. Hyperparameter Tuning:
   - Hyperparameter grids are defined for models that require tuning (SVM, Random Forest, kNN, and Logistic Regression).
   - A grid search combined with stratified 5-fold cross-validation is employed to determine the optimal hyperparameters for each model.

4. Model Evaluation:
   - The best estimator from the grid search for each model is evaluated on the test set.
   - Evaluation metrics include accuracy, precision, recall, F1-score, and ROC-AUC (where applicable), providing a comprehensive assessment of performance.

5. Reproducibility:
   - The use of 'random_state=42' ensures that operations involving randomness (such as data splitting and model initialization) yield consistent results across multiple runs.





In [8]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# Splitting the data: 80% for training/validation, 20% for final testing.
X_train, X_test, y_train, y_test = train_test_split(
    x_reduced, y, test_size=0.2, stratify=y, random_state=42
)

# Define pipelines for each classifier.
pipelines = {
    'svm': Pipeline([
        ('scaler', StandardScaler()),
        ('svm', SVC(probability=True, random_state=42))
    ]),
    'random_forest': Pipeline([
        ('scaler', StandardScaler()),
        ('rf', RandomForestClassifier(random_state=42))
    ]),
    'knn': Pipeline([
        ('scaler', StandardScaler()),
        ('knn', KNeighborsClassifier())
    ]),
    'logistic_regression': Pipeline([
        ('scaler', StandardScaler()),
        ('lr', LogisticRegression(max_iter=1000, random_state=42))
    ]),
    'lda': Pipeline([
        ('scaler', StandardScaler()),
        ('lda', LinearDiscriminantAnalysis())
    ]),
    'qda': Pipeline([
        ('scaler', StandardScaler()),
        ('qda', QuadraticDiscriminantAnalysis())
    ])
}

# Define hyperparameter grids for the models (for those with tunable parameters).
param_grids = {
    'svm': {
        'svm__C': [0.1, 1, 10],
        'svm__kernel': ['linear', 'rbf'],
        'svm__gamma': ['scale', 'auto']
    },
    'random_forest': {
        'rf__n_estimators': [100, 200],
        'rf__max_depth': [None, 10, 20]
    },
    'knn': {
        'knn__n_neighbors': [3, 5, 7],
        'knn__weights': ['uniform', 'distance']
    },
    'logistic_regression': {
        'lr__C': [0.1, 1, 10],
        'lr__penalty': ['l2']
    },
    # LDA and QDA have fewer hyperparameters; grid is left empty.
    'lda': {},
    'qda': {}
}

# Set up stratified k-fold cross-validation.
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

results = {}

# Iterate through each model pipeline.
for model_name, pipeline in pipelines.items():
    print(f"Training model: {model_name}")
    grid_search = GridSearchCV(
        pipeline,
        param_grids.get(model_name, {}),
        cv=cv,
        scoring='accuracy'
    )
    grid_search.fit(X_train, y_train)
    best_model = grid_search.best_estimator_

    # Predict on the test set.
    y_pred = best_model.predict(X_test)

    # Compute evaluation metrics.
    metrics = {
        'accuracy': accuracy_score(y_test, y_pred),
        'precision': precision_score(y_test, y_pred, average='weighted'),
        'recall': recall_score(y_test, y_pred, average='weighted'),
        'f1_score': f1_score(y_test, y_pred, average='weighted')
    }

    # Compute ROC-AUC if the model supports probability estimates.
    try:
        y_proba = best_model.predict_proba(X_test)[:, 1]
        metrics['roc_auc'] = roc_auc_score(y_test, y_proba)
    except Exception as e:
        metrics['roc_auc'] = None

    results[model_name] = metrics

# Output the evaluation results.
print("Evaluation results for all models:")
for model_name, metrics in results.items():
    print(f"{model_name}: {metrics}")


Training model: svm
Training model: random_forest
Training model: knn
Training model: logistic_regression
Training model: lda
Training model: qda
Evaluation results for all models:
svm: {'accuracy': 0.68, 'precision': 0.68, 'recall': 0.68, 'f1_score': 0.68, 'roc_auc': np.float64(0.7727999999999999)}
random_forest: {'accuracy': 0.7, 'precision': 0.7003205128205129, 'recall': 0.7, 'f1_score': 0.6998799519807923, 'roc_auc': np.float64(0.7384000000000001)}
knn: {'accuracy': 0.52, 'precision': 0.52, 'recall': 0.52, 'f1_score': 0.52, 'roc_auc': np.float64(0.5256000000000001)}
logistic_regression: {'accuracy': 0.56, 'precision': 0.56, 'recall': 0.56, 'f1_score': 0.56, 'roc_auc': np.float64(0.6192)}
lda: {'accuracy': 0.5, 'precision': 0.5, 'recall': 0.5, 'f1_score': 0.49979991996798717, 'roc_auc': np.float64(0.5615999999999999)}
qda: {'accuracy': 0.5, 'precision': 0.5, 'recall': 0.5, 'f1_score': 0.49979991996798717, 'roc_auc': np.float64(0.5504)}


/usr/local/lib/python3.11/dist-packages/sklearn/discriminant_analysis.py:1024: LinAlgWarning: The covariance matrix of class 0 is not full rank. Increasing the value of parameter `reg_param` might help reducing the collinearity.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/discriminant_analysis.py:1024: LinAlgWarning: The covariance matrix of class 1 is not full rank. Increasing the value of parameter `reg_param` might help reducing the collinearity.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/discriminant_analysis.py:1024: LinAlgWarning: The covariance matrix of class 0 is not full rank. Increasing the value of parameter `reg_param` might help reducing the collinearity.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/discriminant_analysis.py:1024: LinAlgWarning: The covariance matrix of class 1 is not full rank. Increasing the value of parameter `reg_param` might help reducing the collinearity.
  warnings.warn(
/usr/local/lib/p

## 5. Model evaluation